In [4]:
# pytorch mlp for regression
from numpy import vstack
from numpy import sqrt
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear, Sequential
from torch.nn import Sigmoid , ReLU
from torch.nn import Module
from torch.optim import SGD
from torch.nn import MSELoss
from torch.nn.init import xavier_uniform_
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
import tensorflow as tf

In [5]:
import torch.nn as nn
# dataset definition
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        df = read_csv(path, header=None)
       # df.rename(columns=df.iloc[0]).drop(df.index[0])
        df.drop_duplicates(keep=False,inplace=True)
  
        # store the inputs and outputs
        self.X = df.values[1:, 0:12].astype('float32')
        self.y = df.values[1:, 12:14].astype('float32')
    
        # ensure target has the right shape
        self.y = self.y.reshape((len(self.y), 2))

    # number of rows in the dataset
    def __len__(self):
        return len(self.X)

    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

    # get indexes for train and test rows
    def get_splits(self, n_test=0.2):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])

# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 12)
        #xavier_uniform_(self.hidden1.weight)
        self.act1 = Sigmoid()
        # second hidden layer
        self.hidden2 = Linear(12, 6)
        #xavier_uniform_(self.hidden2.weight)
        self.act2 = Sigmoid()
        # third hidden layer and output
        self.hidden3 = Linear(6, 2)
        #xavier_uniform_(self.hidden3.weight)

    # forward propagate input
    def forward(self, X):
         #input to first hidden layer
         X = self.hidden1(X)
         X = self.act1(X)
         # second hidden layer
         X = self.hidden2(X)
         X = self.act2(X)
         # third hidden layer and output
         X = self.hidden3(X)
         return X

# prepare the dataset
def prepare_data(path):
    # load the dataset
    dataset = CSVDataset(path)
    # calculate split
    train, test = dataset.get_splits()
    
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=1, shuffle=False)
    test_dl = DataLoader(test, batch_size=1, shuffle=False)
    return train_dl, test_dl


# train the model
def train_model(train_dl, model):

    running_loss=0
    correct=0
    total=0
    train_accu = []
    test_loss = 0
    train_losses = []
    # define the optimization
    criterion = MSELoss()
    #loss_fn=nn.CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
   
    # enumerate epochs
    for epoch in range(200):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            test_loss += loss
            #loss=loss_fn(yhat,targets)
            running_loss += loss.item()
            history = loss.item
            total += targets.size(0)
            #print(loss)
            # credit assignment
            loss.backward()
            _, predicted = yhat.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            # update model weights
            optimizer.step()
            if i % 100 == 0:
                #print(f'Train Epoch: {epoch} [{batch_idx*32}/{len(federate_train_loader)*32} ({100. * batch_idx / len(federated_train_loader)}%)] \t Loss: {loss.item()}')
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        epoch, 
                        i * 1 , # no of images done
                        len(train_dl) * 1, # total images left
                        100. * i / len(train_dl), 
                        loss.item()
                     )
                )
          
        train_loss=running_loss/len(train_dl)
        accu=100.*correct/total
        train_accu.append(accu)
        train_losses.append(train_loss)
    test_loss /= len(train_dl)
    print('Train Loss: %.3f | Accuracy: %.3f'%(train_loss,accu))
            

# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        output = model(inputs)
        # retrieve numpy array
        output = output.detach().numpy()


        actual = targets.numpy()
        actual = actual.reshape((len(actual), 2))
        # store
        predictions.append(output)
        actuals.append(actual)
     
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate mse
    mse = mean_squared_error(actuals, predictions)
    #final_train_acc = train_acc/len(test_dl)
    #acc = accuracy_score(actuals, predictions)
    #print(acc)
    return mse

# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

In [6]:
import numpy as np
path = './data/qoe/pursue/simulated_qoe.csv'
train_dl, test_dl = prepare_data(path)


print(len(train_dl.dataset), len(test_dl.dataset))
# define the network
model = MLP(12)
#model = NeuralNetwork()
# train the model
train_model(train_dl, model)
# evaluate the model
mse = evaluate_model(test_dl, model)
rmse = sqrt(mse)
print('MSE: %.3f, RMSE: %.3f' % (mse, sqrt(mse)))
#make a single prediction (expect class=1)
print(rmse)



246 62
Train Epoch: 0 [0/246 (0%)]	Loss: 7.582303
Train Epoch: 0 [100/246 (41%)]	Loss: 0.085501
Train Epoch: 0 [200/246 (81%)]	Loss: 0.085274
Train Epoch: 1 [0/246 (0%)]	Loss: 1.589953
Train Epoch: 1 [100/246 (41%)]	Loss: 0.080303
Train Epoch: 1 [200/246 (81%)]	Loss: 0.085048
Train Epoch: 2 [0/246 (0%)]	Loss: 1.605685
Train Epoch: 2 [100/246 (41%)]	Loss: 0.079630
Train Epoch: 2 [200/246 (81%)]	Loss: 0.084846
Train Epoch: 3 [0/246 (0%)]	Loss: 1.623642
Train Epoch: 3 [100/246 (41%)]	Loss: 0.078802
Train Epoch: 3 [200/246 (81%)]	Loss: 0.084593
Train Epoch: 4 [0/246 (0%)]	Loss: 1.656591
Train Epoch: 4 [100/246 (41%)]	Loss: 0.077351
Train Epoch: 4 [200/246 (81%)]	Loss: 0.083657
Train Epoch: 5 [0/246 (0%)]	Loss: 1.754469
Train Epoch: 5 [100/246 (41%)]	Loss: 0.073408
Train Epoch: 5 [200/246 (81%)]	Loss: 0.076276
Train Epoch: 6 [0/246 (0%)]	Loss: 2.005519
Train Epoch: 6 [100/246 (41%)]	Loss: 0.067611
Train Epoch: 6 [200/246 (81%)]	Loss: 0.067616
Train Epoch: 7 [0/246 (0%)]	Loss: 2.065365
Train

Train Epoch: 60 [0/246 (0%)]	Loss: 2.074271
Train Epoch: 60 [100/246 (41%)]	Loss: 0.064829
Train Epoch: 60 [200/246 (81%)]	Loss: 0.064354
Train Epoch: 61 [0/246 (0%)]	Loss: 2.074271
Train Epoch: 61 [100/246 (41%)]	Loss: 0.064825
Train Epoch: 61 [200/246 (81%)]	Loss: 0.064357
Train Epoch: 62 [0/246 (0%)]	Loss: 2.074274
Train Epoch: 62 [100/246 (41%)]	Loss: 0.064821
Train Epoch: 62 [200/246 (81%)]	Loss: 0.064361
Train Epoch: 63 [0/246 (0%)]	Loss: 2.074279
Train Epoch: 63 [100/246 (41%)]	Loss: 0.064817
Train Epoch: 63 [200/246 (81%)]	Loss: 0.064365
Train Epoch: 64 [0/246 (0%)]	Loss: 2.074282
Train Epoch: 64 [100/246 (41%)]	Loss: 0.064813
Train Epoch: 64 [200/246 (81%)]	Loss: 0.064368
Train Epoch: 65 [0/246 (0%)]	Loss: 2.074286
Train Epoch: 65 [100/246 (41%)]	Loss: 0.064808
Train Epoch: 65 [200/246 (81%)]	Loss: 0.064371
Train Epoch: 66 [0/246 (0%)]	Loss: 2.074293
Train Epoch: 66 [100/246 (41%)]	Loss: 0.064804
Train Epoch: 66 [200/246 (81%)]	Loss: 0.064375
Train Epoch: 67 [0/246 (0%)]	Loss:

Train Epoch: 119 [200/246 (81%)]	Loss: 0.064397
Train Epoch: 120 [0/246 (0%)]	Loss: 2.075104
Train Epoch: 120 [100/246 (41%)]	Loss: 0.064443
Train Epoch: 120 [200/246 (81%)]	Loss: 0.064395
Train Epoch: 121 [0/246 (0%)]	Loss: 2.075123
Train Epoch: 121 [100/246 (41%)]	Loss: 0.064436
Train Epoch: 121 [200/246 (81%)]	Loss: 0.064394
Train Epoch: 122 [0/246 (0%)]	Loss: 2.075143
Train Epoch: 122 [100/246 (41%)]	Loss: 0.064429
Train Epoch: 122 [200/246 (81%)]	Loss: 0.064392
Train Epoch: 123 [0/246 (0%)]	Loss: 2.075162
Train Epoch: 123 [100/246 (41%)]	Loss: 0.064422
Train Epoch: 123 [200/246 (81%)]	Loss: 0.064390
Train Epoch: 124 [0/246 (0%)]	Loss: 2.075184
Train Epoch: 124 [100/246 (41%)]	Loss: 0.064415
Train Epoch: 124 [200/246 (81%)]	Loss: 0.064388
Train Epoch: 125 [0/246 (0%)]	Loss: 2.075203
Train Epoch: 125 [100/246 (41%)]	Loss: 0.064408
Train Epoch: 125 [200/246 (81%)]	Loss: 0.064386
Train Epoch: 126 [0/246 (0%)]	Loss: 2.075223
Train Epoch: 126 [100/246 (41%)]	Loss: 0.064401
Train Epoch: 

Train Epoch: 179 [0/246 (0%)]	Loss: 2.073833
Train Epoch: 179 [100/246 (41%)]	Loss: 0.065222
Train Epoch: 179 [200/246 (81%)]	Loss: 0.064182
Train Epoch: 180 [0/246 (0%)]	Loss: 2.073838
Train Epoch: 180 [100/246 (41%)]	Loss: 0.065221
Train Epoch: 180 [200/246 (81%)]	Loss: 0.064180
Train Epoch: 181 [0/246 (0%)]	Loss: 2.073841
Train Epoch: 181 [100/246 (41%)]	Loss: 0.065220
Train Epoch: 181 [200/246 (81%)]	Loss: 0.064178
Train Epoch: 182 [0/246 (0%)]	Loss: 2.073846
Train Epoch: 182 [100/246 (41%)]	Loss: 0.065220
Train Epoch: 182 [200/246 (81%)]	Loss: 0.064176
Train Epoch: 183 [0/246 (0%)]	Loss: 2.073851
Train Epoch: 183 [100/246 (41%)]	Loss: 0.065219
Train Epoch: 183 [200/246 (81%)]	Loss: 0.064174
Train Epoch: 184 [0/246 (0%)]	Loss: 2.073854
Train Epoch: 184 [100/246 (41%)]	Loss: 0.065218
Train Epoch: 184 [200/246 (81%)]	Loss: 0.064173
Train Epoch: 185 [0/246 (0%)]	Loss: 2.073861
Train Epoch: 185 [100/246 (41%)]	Loss: 0.065217
Train Epoch: 185 [200/246 (81%)]	Loss: 0.064171
Train Epoch: 

In [7]:
row1 = [1,1,1,1,172423,4300,30,0,6,10,900,0.9] #0,6
row2 = [1,1,1,1,538,1850,12,0,6,10,120,0.12]#,5,1
row3 = [1,1,1,1,53463,750,4,11,6,10,20,0.02]#,3,1

yhat = predict(row3, model)
print('Predicted:',yhat)

Predicted: [[6.023859  0.7723446]]


In [8]:
import torch 
torch.save(model.state_dict(), './model/model_central_training.pt')

In [9]:
def loadmodel(path):
    model = MLP(12)
    model.load_state_dict(torch.load(path))
    model.eval()
    return model

In [10]:
model_path = './model/model_central_training.pt'
model_loaded = loadmodel(model_path)

In [11]:
print(model_loaded)

MLP(
  (hidden1): Linear(in_features=12, out_features=12, bias=True)
  (act1): Sigmoid()
  (hidden2): Linear(in_features=12, out_features=6, bias=True)
  (act2): Sigmoid()
  (hidden3): Linear(in_features=6, out_features=2, bias=True)
)


In [12]:
yhat = predict(row2, model_loaded)
print('Predicted:',yhat)

Predicted: [[6.023859  0.7723446]]


In [13]:
yhat[0][1]

0.7723446

In [14]:
yhat[0][0]

6.023859